In [ ]:
"""
This file is part of lc-power-match-baluns.
Copyright © 2023 Technical University of Denmark (developed by Rasmus Jepsen)

This library is free software; you can redistribute it and/or
modify it under the terms of the GNU Lesser General Public
License as published by the Free Software Foundation; either
version 2.1 of the License, or (at your option) any later version.

This library is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public
License along with this library; if not, write to the Free Software
Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301  USA
"""

In [ ]:
"""
This notebook shows the process that was used to determine the design equations for the novel lattice power matching LC balun topology.

Symmetric lattice baluns have previously been used for real-real matching [1],
though they require extra elements if used for complex impedance matching.
This topology relaxes the symmetry constraint to provide complex impedance matching with four elements.
Apel and Page [2] have also previously considered a balun network that resembles a specific case of an asymmetric lattice balun.
"""

In [ ]:
# import modules
from lcapy import Circuit, oo, expr, symbol, j, Eq, Z, Matrix, limit, one
from lcapy.expr import symbols

In [ ]:
# create lcapy circuit from netlist
# the balanced port is between nodes 1 and 3, and the unbalanced port is between nodes 2 and 0
balun_cct = Circuit("""
Z2 1 2_0; right
W 2_0 2; right
Z4 2 3; rotate=225
W 3 3_0; rotate=225
Z3 3_0 0; right
W 1_0 1; right
Z1 1_0 0_0; rotate=-45
W 0_1 0; rotate=-45
W 0_0 0_1; right
""")
balun_cct.draw()

In [ ]:
# create a differential mode two-port model
balun_twoport = balun_cct.twoport(1, 3, 2, 0)

In [ ]:
# retrieve the three-port Z parameters
balun_threeport_z = balun_cct.Zparamsn(2,0,1,0,3,0)
balun_threeport_z

In [ ]:
# initialise symbols

# resistances of the balanced and unbalanced ports
r_b, r_u = symbols('R_B R_U', real=True)

# reactances of the balanced and unbalanced ports
x_b, x_u = symbols('X_B X_U', real=True)

# impedances of the balanced and unbalanced ports
z_b, z_u = symbols('Z_B Z_U', complex=True)

# element reactances
x_1, x_2, x_3, x_4 = symbols('X_1 X_2 X_3 X_4', real=True)

In [ ]:
# The technique described in [3] is used to convert the three-port impedance matrix to three-port scattering parameters.

In [ ]:
# initialise matrices for converting Z-parameters to S-parameters

f = Matrix(((z_u.real ** 0.5 / 2, 0, 0), (0, (z_b.real / 2) ** 0.5 / 2, 0), (0, 0, (z_b.real / 2) ** 0.5 / 2)))

g = Matrix(((z_u, 0, 0), (0, z_b / 2, 0), (0, 0, z_b / 2)))

g_plus = Matrix(((z_u.conj, 0, 0), (0, z_b.conj / 2, 0), (0, 0, z_b.conj / 2)))

In [ ]:
# calculate the renormalised three-port scattering parameters
balun_threeport_s = f * (balun_threeport_z - g_plus) * (balun_threeport_z + g).inv() * f.inv()
balun_threeport_s

In [ ]:
# The method described in [4] is used to calculate the common-mode rejection ratio (CMRR).

In [ ]:
# calculate common-mode response
balun_cct_s21cs = expr('1/sqrt(2)') * (balun_threeport_s[1,0] + balun_threeport_s[2,0])
balun_cct_s21cs.simplify()

In [ ]:
# calculate differential-mode response
balun_cct_s21ds = expr('1/sqrt(2)') * (balun_threeport_s[1,0] - balun_threeport_s[2,0])
balun_cct_s21ds.simplify()

In [ ]:
# variable substitutions for later steps
substitutions = {'Z1': j * x_1, 'Z2': j * x_2, 'Z3': j * x_3, 'Z4': j * x_4, 'Z_B': r_b + j * x_b, 'Z_U': r_u + j * x_u}

In [ ]:
# find the inverse of the CMRR (this should be 0 for an ideal balun)
inv_cmrr = (balun_cct_s21cs / balun_cct_s21ds).subs(substitutions).simplify()
inv_cmrr

In [ ]:
# solve for element reactances that achieve infinite CMRR
# the numerator of inv_cmrr is used as this makes the solver more reliable
cmrr_system = expr([Eq(inv_cmrr.N, 0)])
cmrr_unknowns = expr((x_1, x_2, x_3, x_4))
cmrr_solutions = cmrr_system.solve(cmrr_unknowns)
cmrr_solutions

In [ ]:
# substitute the values back into inv_cmrr to check
# because cmrr_solutions[2] is the only solution that yields an inv_cmrr of 0, it is the only valid solution
inv_cmrr_subbed = list(inv_cmrr.subs(list((unknown, sol[i]) for i, unknown in enumerate(cmrr_unknowns))).simplify() for sol in cmrr_solutions)
inv_cmrr_subbed

In [ ]:
# find the impedance parameters of the two-port model of the network
twoport_z = balun_twoport.Zparams.subs(substitutions).simplify()
twoport_z

In [ ]:
# find the impedance parameters of the two-port model of the network
twoport_z_subbed = twoport_z.subs(x_1, cmrr_solutions[2][0]).subs(x_2, cmrr_solutions[2][1]).simplify()
twoport_z_subbed

In [ ]:
"""Using the impedance parameters, the remaining element reactances will be solved such that power matching is achieved.
For more details, see conjugate_matching_z_params.ipynb
"""

In [ ]:
# initialise symbols for reactance parameters
x_11, x_12, x_22 = symbols('X_11 X_12 X_22', real=True)

In [ ]:
system_1 = expr([Eq(twoport_z_subbed[0, 0].imag.simplify(), x_11),
                 Eq(twoport_z_subbed[0, 1].imag.simplify() ** 2, (r_u * (r_b ** 2 + x_11 ** 2 + 2 * x_11 * x_b + x_b ** 2) / r_b)),
                 Eq(twoport_z_subbed[1, 1].imag.simplify(), (r_u * x_11 + r_u * x_b - r_b * x_u) / r_b)])
unknowns_1 = expr((x_3, x_4, x_11))
solutions_1 = system_1.solve(unknowns_1)
solutions_1

In [ ]:
# The final design equations are presented below:
# Note that the two solutions result in equivalent networks, with X1 and X3, and X2 and X4 being swapped.
# Therefore, only one solution is presented in the paper.
# These equations have been further simplified in the paper.

In [ ]:
# x_1 first solution
cmrr_solutions[2][0].subs(x_3, solutions_1[0][0]).subs(x_4, solutions_1[0][1]).simplify()

In [ ]:
# x_1 second solution
cmrr_solutions[2][0].subs(x_3, solutions_1[1][0]).subs(x_4, solutions_1[1][1]).simplify()

In [ ]:
# x_2 first solution
cmrr_solutions[2][1].subs(x_3, solutions_1[0][0]).subs(x_4, solutions_1[0][1]).simplify()

In [ ]:
# x_2 second solution
cmrr_solutions[2][1].subs(x_3, solutions_1[1][0]).subs(x_4, solutions_1[1][1]).simplify()

In [ ]:
# x_3 first solution
solutions_1[0][0].simplify()

In [ ]:
# x_3 second solution
solutions_1[1][0].simplify()

In [ ]:
# x_4 first solution
solutions_1[0][1].simplify()

In [ ]:
# x_4 second solution
solutions_1[1][1].simplify()

In [ ]:
"""References:
[1] C Lorenz AG, "Circuit arrangement for the transition from a symmetrical electrical arrangement to an asymmetrical one, in particular in the case of high-frequency arrangements," Germany Patent 603 816, April 1, 1932. [Online]. Available: https://patents.google.com/patent/DE603816C/en
[2] T. R. Apel and C. E. Page, "Lumped parameter balun," English, pat. 5 574 411, 1995. [Online]. Available: https://patents.google.com/patent/US5574411A/en.
[3] K. Kurokawa, "Power waves and the scattering matrix," IEEE Transactions on Microwave Theory and Techniques, vol. 13, no. 2, pp. 194–202, 1965.
[4] D. Bockelman and W. Eisenstadt, "Combined differential and common-mode analysis of power splitters and combiners," IEEE Transactions on Microwave Theory and Techniques, vol. 43, no. 11, pp. 2627–2632, 1995.
"""